In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import the geoapify_key variable from api_keys.py
from api_keys import geoapify_key

# Remove  warnings related to the CryptographyDeprecationWarning
import warnings
from cryptography.utils import CryptographyDeprecationWarning

warnings.filterwarnings("ignore", category=CryptographyDeprecationWarning)

In [2]:
# Load the CSV file created in WeatherPy.ipynb into a Pandas DataFrame
csv_file = Path("..", "Output", "cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,petropavlovsk-kamchatsky,53.0452,158.6483,-10.67,48,40,8.00,RU,1708442249
1,1,stanley,54.8680,-1.6985,11.39,82,75,5.66,GB,1708442250
2,2,adamstown,-25.0660,-130.1015,25.72,84,100,5.39,PN,1708442251
3,3,coyhaique,-45.5752,-72.0662,21.34,56,75,2.57,CL,1708442252
4,4,ushuaia,-54.8000,-68.3000,10.79,57,40,9.26,AR,1708442252


In [9]:
# Initialize the map
map = city_data_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo=True, 
    tiles='OSM', 
    frame_width=800, 
    frame_height=600,
    xticks=[0], 
    yticks=list(range(-80, 81, 20)), 
    size='Humidity', 
    color='City', 
    legend='right'
)

# Display the map
map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [4]:
# Filter the DataFrame based on parameters adjust to fit criteria
ideal_weather_df = city_data_df[(city_data_df['Max Temp'] < 27) & 
                                (city_data_df['Max Temp'] > 21) & 
                                (city_data_df['Wind Speed'] < 4.5) & 
                                (city_data_df['Cloudiness'] == 0)]


ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,cabo san lucas,22.8909,-109.9124,22.09,47,0,3.09,MX,1708442157
86,86,al hufuf,25.3646,49.5653,22.19,30,0,2.57,SA,1708442330
267,267,acapulco de juarez,16.8634,-99.8901,24.90,73,0,1.54,MX,1708442157
277,277,mazatlan,23.2167,-106.4167,22.17,67,0,0.45,MX,1708442392
373,373,lazaro cardenas,17.9583,-102.2000,24.83,65,0,1.33,MX,1708442242
376,376,las cabezas de san juan,36.9838,-5.9393,23.47,47,0,2.71,ES,1708442521
390,390,brisas de zicatela,15.8369,-97.0419,26.29,61,0,4.12,MX,1708442532
392,392,navalmoral de la mata,39.8916,-5.5406,23.87,45,0,0.86,ES,1708442533
431,431,haiku-pauwela,20.9219,-156.3051,21.13,74,0,3.60,US,1708442565
487,487,pisco,-13.7000,-76.2167,26.03,78,0,3.60,PE,1708442603


In [5]:
# Selecting desired columns
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Adding an empty column
hotel_df = hotel_df.assign(**{"Hotel Name": ""})

hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
5,cabo san lucas,MX,22.8909,-109.9124,47,
86,al hufuf,SA,25.3646,49.5653,30,
267,acapulco de juarez,MX,16.8634,-99.8901,73,
277,mazatlan,MX,23.2167,-106.4167,67,
373,lazaro cardenas,MX,17.9583,-102.2000,65,
376,las cabezas de san juan,ES,36.9838,-5.9393,47,
390,brisas de zicatela,MX,15.8369,-97.0419,61,
392,navalmoral de la mata,ES,39.8916,-5.5406,45,
431,haiku-pauwela,US,20.9219,-156.3051,74,
487,pisco,PE,-13.7000,-76.2167,78,


In [6]:
# Set parameters to search for a hotel
radius = 10000  # Define your radius here
params = {
    "categories": "accommodation.hotel",  # Include only accommodation category
    "filter": "",   # Filter places by bounds, circle, geometry, or countries
    "bias": "",     # Search first near the location
    "apiKey": geoapify_key   # Required parameter for API key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Construct the API URL
    api_url = "https://api.geoapify.com/v2/places?" + "&".join([f"{key}={value}" for key, value in params.items()])
    
    # Print the API URL
    print("API URL:", api_url)
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Print the response status code
    print("Response status code:", response.status_code)
    
    # Print the response content
    print("Response content:", response.text)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
API URL: https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=circle:-109.9124,22.8909,10000&bias=proximity:-109.9124,22.8909&apiKey=f766e8800bec434ab962625d6296e87c
Response status code: 200
Response content: {"type":"FeatureCollection","features":[{"type":"Feature","properties":{"name":"Comfort Rooms","country":"Mexico","country_code":"mx","state":"Baja California Sur","county":"Los Cabos Municipality","city":"Cabo San Lucas","postcode":"23469","suburb":"Juárez","street":"Calle Revolución de 1910","lon":-109.9119612,"lat":22.8904798,"state_code":"BCS","formatted":"Comfort Rooms, Calle Revolución de 1910, Juárez, 23469 Cabo San Lucas, BCS, Mexico","address_line1":"Comfort Rooms","address_line2":"Calle Revolución de 1910, Juárez, 23469 Cabo San Lucas, BCS, Mexico","categories":["accommodation","accommodation.hotel"],"details":[],"datasource":{"sourcename":"openstreetmap","attribution":"© OpenStreetMap contributors","license":"Open Database Lice

,City,Country,Lat,Lng,Humidity,Hotel Name
5,cabo san lucas,MX,22.8909,-109.9124,47,Comfort Rooms
86,al hufuf,SA,25.3646,49.5653,30,Al Muhaidab Residence Al Ahsa
267,acapulco de juarez,MX,16.8634,-99.8901,73,Hotel del Valle
277,mazatlan,MX,23.2167,-106.4167,67,Mona Inn
373,lazaro cardenas,MX,17.9583,-102.2000,65,Hotel Sol del Pacífico
376,las cabezas de san juan,ES,36.9838,-5.9393,47,Venta El Pan
390,brisas de zicatela,MX,15.8369,-97.0419,61,Casa de Olas
392,navalmoral de la mata,ES,39.8916,-5.5406,45,Hotel Moya
431,haiku-pauwela,US,20.9219,-156.3051,74,Inn At Mama's Fish House
487,pisco,PE,-13.7000,-76.2167,78,La Portada


In [10]:
# Create the map
hotel_map = hotel_df.hvplot.points(
    'Lng', 
    'Lat',
    geo=True, 
    tiles='OSM', 
    frame_width=800, 
    frame_height=600,
    xticks=[0], 
    yticks=list(range(-80, 81, 20)), 
    size='Humidity', 
    color='City',
    legend='top_right',
    hover_cols=['City', 'Country', 'Lat', 'Lng', 'Humidity', 'Hotel Name']
)

# Display the map
hotel_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)